## Basic Areas Tutorial
<a target="_blank" href="https://colab.research.google.com/github/VirtualBrainLab/urchin-examples/blob/colab/basics/areas.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## Run the following block if using the notebook for the first time:

Urchin is a python package that needs to properly be installed before use the first time. Once installed within the environment, it only needs to be imported. If running locally, Urchin only needs to be installed once when creating the virtual environment. For more information about Urchin, check out the documentation on the VBL website [here](https://virtualbrainlab.org/urchin/installation_and_use.html).

In [ ]:
#Installing urchin
!pip install oursin -U

## Set up the Unity Renderer for Neuroscience and open the renderer webpage 
Ensure that popups are enabled BEFORE running the following code, to allow Urchin to properly open in a new window. You can also visit the renderer site once Urchin is imported at https://data.virtualbrainlab.org/Urchin/?ID=[INSERT ID OUTPUTTED BELOW]

In [ ]:
#Importing necessary libraries:
import oursin as urchin
urchin.setup()

## Brain areas tutorial below

This is a tutorial for displaying different brain areas, and setting different properties. This is one of the more basic but important features.

In the code below, 'grey' is referring to grey matter, or the entire brain. set_visibility makes the area appear, and set_material dictates how it displays. set_material takes lighting and transparency as input variables, and the options are 'opaque-lit', 'opaque-unlit','transparent-lit' and 'transparent-unlit'. The alpha value further adjusts the transparency.

In [ ]:
# load a single area
urchin.ccf.set_visibility({'grey':True})
urchin.ccf.set_material({'grey':'transparent-unlit'})
urchin.ccf.set_alpha({'grey':0.5})

The clearing function clears everything on the screen.

In [ ]:

urchin.clear()

If you want to load just one hemisphere at a time, append either the -lh (left hand) or -rh (right hand) suffix to the area(s) loaded.

In [ ]:
# load just left hemisphere

urchin.ccf.set_visibility({'grey-lh':True})
urchin.ccf.set_material({'grey-lh':'transparent-unlit'})
urchin.ccf.set_alpha({'grey-lh':0.5})

# same pattern for right hemisphere with "-rh" suffix

In [ ]:

urchin.clear()

To load the beryl areas, there is a sepearate function to do so.

In [ ]:
# load all the beryl areas
urchin.ccf.load_beryl()